In [ ]:
!pip install python-dotenv langchain pinecone-client

In [ ]:
!pip install langchain-pinecone

In [ ]:
!pip install langchain-google-genai

In [ ]:
import os

os.environ["GOOGLE_API_KEY"] = "GOOGLE_API_KEY"
os.environ["PINECONE_API_KEY"] = "PINECONE_API_KEY"
os.environ["PINECONE_ENV"] = "PINECONE_ENV"
os.environ["PINECONE_INDEX_NAME"] = "PINECONE_INDEX_NAME"
os.environ["RAPID_API_KEY"] = "RAPID_API_KEY"

In [15]:
import os
from dotenv import load_dotenv
from langchain_pinecone import PineconeVectorStore
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from pinecone import Pinecone as PineconeClient
import requests

# Load env variables
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENV = os.getenv("PINECONE_ENV")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
RAPID_API_KEY = os.getenv("RAPID_API_KEY")

# Ensure Gemini API key is set
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# Embeddings using Gemini
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Pinecone setup
# Make sure PINECONE_API_KEY is your actual API key
pinecone = PineconeClient(api_key=PINECONE_API_KEY, environment=PINECONE_ENV)
# Make sure PINECONE_INDEX_NAME is your actual index name
index = pinecone.Index(PINECONE_INDEX_NAME)

# Vector store
vectorstore = PineconeVectorStore(index=index, embedding=embeddings, text_key="text")

# LLM (Gemini)
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2)

# Memory
memory = ConversationBufferMemory(
    return_messages=True,
    input_key="question",
    memory_key="chat_history"
)
# Function to fetch tech jobs in a city (using RapidAPI or another job API)
def get_tech_jobs(city):
    url = "https://jsearch.p.rapidapi.com/search"
    querystring = {"query": f"tech jobs in {city}", "page": "1"}
    headers = {
        "X-RapidAPI-Key": RAPID_API_KEY,
        "X-RapidAPI-Host": "jsearch.p.rapidapi.com"
    }
    response = requests.get(url, headers=headers, params=querystring)
    data = response.json()
    jobs = data.get("data", [])
    return "\n".join([f"{job['job_title']} at {job['employer_name']}" for job in jobs[:3]]) if jobs else "No jobs found."

def get_cost_of_living(city):
    # Placeholder for Numbeo or static cost-of-living data
    return f"The cost of living in {city} is moderate to high depending on the neighborhood."


# QA chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    memory=memory,
)
# Save previous important user queries & answers as embedded docs

# 💬 Interactive loop
print("🤖 Ask me anything! Type 'exit' to end the conversation.\n")

while True:
    user_input = input("🧑 You: ")

    if user_input.lower() in ["exit", "quit"]:
        print("👋 Goodbye!")
        break

    # ✅ Check for real-time tool usage
    if "tech jobs" in user_input.lower():
        city = user_input.split("in")[-1].strip() if "in" in user_input else "San Francisco"
        print("🛠 Fetching tech jobs...")
        print(get_tech_jobs(city))
        continue

    elif "cost of living" in user_input.lower():
        city = user_input.split("in")[-1].strip() if "in" in user_input else "New York"
        print("🛠 Fetching cost of living...")
        print(get_cost_of_living(city))
        continue

    # ✅ Inject long-term memory into context
    past_memories = vectorstore.similarity_search(user_input, k=3, filter={"type": "memory"})
    if past_memories:
        print("📚 Recalling related past chats...")
        for mem in past_memories:
            print("🧠", mem.page_content)

    # 🧠 Ask Gemini
    response = qa_chain.invoke({"question": user_input})
    print(f"🤖 Gemini: {response['answer']}")

    important_memory = f"User asked: {user_input}, Gemini said: {response['answer']}"
vectorstore.add_texts([important_memory], metadata={"type": "memory", "user": "u1"})
past_memories = vectorstore.similarity_search("tech jobs", k=5, filter={"type": "memory", "user": "u1"})
for memory in past_memories:
    print(memory.page_content)

    # ✅ Store to long-term memory
    memory_log = f"User asked: {user_input}\nGemini said: {response['answer']}"
    vectorstore.add_texts([memory_log], metadata={"type": "memory", "user": "u1"})

    # Optional: print session memory
    print("\n🧠 Chat so far:")
    for msg in memory.chat_memory.messages:
        role = "You" if msg.type == "human" else "Gemini"
        print(f"{role}: {msg.content}")
    print("-" * 50)



🤖 Ask me anything! Type 'exit' to end the conversation.

🧑 You: 2+2?
🤖 Gemini: 4
🧑 You: What's the cost of living in NewYork?
🛠 Fetching cost of living...
The cost of living in NewYork? is moderate to high depending on the neighborhood.
🧑 You: 3-1?
🤖 Gemini: 2
🧑 You: what i asked before?
🤖 Gemini: Previously, you asked:

* "previous answer-2?"
* "4-5?"
* "2+2?" (twice)
🧑 You: exit
👋 Goodbye!
